<a href="https://colab.research.google.com/github/CristianS1988/GooSL/blob/main/ProyectoModeladoSig.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<figure>
<center>
<img src='https://postgrado.ingenieria.usac.edu.gt/wp-content/uploads/2022/08/eep-logob-2048x450.png'width="550" height="120"/>
<center/>
</figure>

#Problema Central
* Subestimación de concentraciones de los gases contaminantes en la calidad de aire en el municipio de San Andrés, departamento de Petén de marzo del año 2022.


# Pregunta Principal

* ¿Cuál es el área con mayor concentración de gases contaminantes en el municipio de San Andrés departamento de Petén.?

#Objetivo General
* Determinar las concentraciones dióxido de nitrógeno (NO2) que se encuentren en el aire  a través de sensor Tropomi


#Insumos  
* Imagen Sentinel 5 duratente el mes de marzo del año 2022 en temporada de verano  (incendios forestales)
* Datos de puntos de calor de CONABIO del mes de marzo del año 2022
* Datos de incendios forestales de NASA FIRMS

* Recopilacion de datos de imagenes sentinel, y puntos de calor de NASA FIRMS
* Obtención de datos (filtrado de datos durante el mes de marzo del año 2022)
* Variable: Concentración de gas
# Método
* Análisis de .....

Instalación de todos los paquetes necesarios para realizar el modelado

In [1]:
!pip install geopandas

In [2]:
!pip install geemap


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 81.1 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11124 sha256=67075d425c4576ede546ed6ba15b41c923ce05315c0d714db3d96b23522eab11
  Stored i

In [3]:
!pip install -U geemap


In [4]:
!pip show nombre_biblioteca

In [5]:
!pip install earthengine-api


Inicio y autenticación de usuario para conexión a GEE

In [6]:
# Importar Earth Engine y Folium
import ee  #EartEngine
import folium #Visualizador de mapas

In [7]:
#Autentificar e iniciar Earth Engine
# Iniciar el flujo de autenticación.
ee.Authenticate()

# Inicializar la biblioteca.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=APkRt9aygUVgIri0pHCQpbG0-6MxSIMSJxInIn7yz_g&tc=CQnJ_RJUKRmWWJ_47sOw3FMqUk2CrA3h6hVm92JRnsg&cc=Lmta_4klxPnfOO-qumyjL7bL83Vc8UC95O4LVLrDSGs

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AZEOvhX21CK3iHPL3tIdg5WCh59KJsfMTbQcSO4RQ5T5m-NXXpKgnMohcp8

Successfully saved authorization token.


In [8]:
# Configurar Earth Engine para mostrar en un mapa de folium
# Proceso para mostrar imágenes de Earth Engine en un mapa folium
def add_ee_layer(self, ee_object, vis_params, name):

    try:
        # Mostrar ee.Image()
        if isinstance(ee_object, ee.image.Image):
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
                tiles=map_id_dict['tile_fetcher'].url_format,
                attr='Google Earth Engine',
                name=name,
                overlay=True,
                control=True
            ).add_to(self)
    # Mostrar ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
                tiles=map_id_dict['tile_fetcher'].url_format,
                attr='Google Earth Engine',
                name=name,
                overlay=True,
                control=True
            ).add_to(self)
 # Mostrar ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):
            folium.GeoJson(
                data=ee_object.getInfo(),
                name=name,
                overlay=True,
                control=True
            ).add_to(self)

# Mostrar ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
                tiles=map_id_dict['tile_fetcher'].url_format,
                attr='Google Earth Engine',
                name=name,
                overlay=True,
                control=True
            ).add_to(self)

    except:
        print("No se pudo mostrar {}".format(name))

# Agregar el método de dibujo de EE a folium.
folium.Map.add_ee_layer = add_ee_layer

**Delimitación del área de estudio: Municipio de San Andrés, Petén, Guatemala.**

In [9]:

import geemap

# Ruta del asset en Earth Engine
asset_path = "projects/ee-rjoscelyn11/assets/pol_sanandres"

# Cargar el asset como una colección de características
collection = ee.FeatureCollection(asset_path)

# Crear un objeto de mapa
Map = geemap.Map()

# Agregar el polígono al mapa
Map.addLayer(collection, {}, 'Polígono San Andrés')

# Centrar el mapa en el polígono
Map.centerObject(collection)

# Visualizar el mapa
Map


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

**Delimitación de incendios forestales del que corresponden al mes de marzo del año 2022 fuentes de datos: NASA FIRMS**

In [10]:
import datetime
import geemap

# Definir el rango de fechas de interés
fecha_inicio = datetime.datetime(2022, 3, 1)
fecha_fin = datetime.datetime(2022, 3, 31)

# Filtrar los datos de incendios en el área de estudio y el rango de fechas
datos_incendios = ee.ImageCollection('FIRMS').filterDate(fecha_inicio, fecha_fin)

# Definir la región de interés (usando coordenadas delimitantes)
regioninteres = ee.Geometry.Rectangle([-91, 17.20, -89.80, 17.80])

# Realizar el recorte (clip) de los datos de incendios a la región de interés
datos_incendios_recortados = datos_incendios.map(lambda image: image.clip(regioninteres))

# Calcular la cantidad total de incendios en el área de estudio y el rango de fechas
cantidad_incendios = datos_incendios_recortados.size().getInfo()
print('Cantidad de incendios:', cantidad_incendios)

# Crear un mapa interactivo utilizando geemap
mapa = geemap.Map(center=[17.4248, -90.4724], zoom=9)

# Agregar los datos de incendios recortados al mapa
mapa.addLayer(datos_incendios_recortados, {}, 'Datos de incendios recortados')

# Mostrar el mapa interactivo
mapa

Cantidad de incendios: 30


Map(center=[17.4248, -90.4724], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

**Delimitación de puntos de calor correspondientes al mes de marzo del año 2022 de la fuente de datos de:CONABIO**

In [11]:
# Ruta del asset en Earth Engine
asset_path = "projects/ee-rjoscelyn11/assets/puntoscalor"

# Cargar el asset como una colección de características
collection = ee.FeatureCollection(asset_path)

# Visualización en un mapa
Map = geemap.Map()
Map.addLayer(collection, {}, 'Puntos de calor')
Map.centerObject(collection)
Map


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

Ingreso de imangen de Sentinel 5 con datos de monóxido de carbono CO para la región del municipio de San Andrés, Petén del mes de marzo del año 2022

In [12]:
#buscar el municipio de San Andrés, Petén, el cual define el área de interes.
San_Andres = ee.FeatureCollection('projects/ee-rjoscelyn11/assets/pol_sanandres')

#generar el centroide del poligono seleccionado
centroid = San_Andres.geometry().centroid().getInfo()

# Crear un objeto Map en geemap agregando los centroides como punto de partida
Map = geemap.Map(center=[centroid['coordinates'][1], centroid['coordinates'][0]], zoom=9)


Map.addLayer(San_Andres, {}, "San Andres")
Map.addLayerControl()
Map


Map(center=[17.48975378476301, -90.40494849453306], controls=(WidgetControl(options=['position', 'transparent_…

In [13]:
#Corte de concentraciones de Mónoxido de Carbono para el área de interes del municipio de San Andrés, Petén.
#geometry = San_Andres.geometry()
collection = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_CO') \
  .filterDate('2023-03-01', '2023-03-30') \
  .filterBounds(San_Andres)

# Recortar la imagen utilizando la geometría
union = collection.max() #union de las imagenes pero se priorizaron los mayores datos
clip = union.clip(San_Andres)

band_viz = {
    'min': 0,
    'max': 0.05,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

# Agregar la capa de CO al mapa
Map.addLayer(clip.select('CO_column_number_density'), band_viz, 'S5P CO - Marzo 2022')
Map

Map(bottom=59367.0, center=[17.48975378476301, -90.40494849453306], controls=(WidgetControl(options=['position…

In [14]:

# Definir la proyección objetivo (WGS84)
proyeccion_objetivo = 'EPSG:4326'

# Fecha de interés
fecha_inicio = '2022-03-01'
fecha_fin = '2022-03-31'

# Cargar la colección de imágenes filtrada por fecha
coleccion = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_CO').filterDate(fecha_inicio, fecha_fin)

# Seleccionar una imagen de la colección
imagen_original = coleccion.first()

# Reproyectar la imagen a WGS84
imagen_reproyectada = imagen_original.reproject(proyeccion_objetivo)

# Visualizar la imagen reproyectada
Map.addLayer(imagen_reproyectada, {}, 'Imagen reproyectada a WGS84')


In [15]:
# Obtener la información de proyección de la imagen reproyectada
proyeccion = imagen_reproyectada.projection()

# Imprimir la información de proyección
print('Información de proyección:')
print('Código de referencia espacial:', proyeccion.crs().getInfo())



Información de proyección:
Código de referencia espacial: EPSG:4326


In [17]:
import geemap
import datetime
import ee

# Inicializar Google Earth Engine
ee.Initialize()

# Leer la tabla de Earth Engine
table = ee.FeatureCollection("projects/ee-rjoscelyn11/assets/pol_sanandres")
geometry = table.geometry()  # Obten la geometría

# Definir la fecha inicial y final del mes
start_date = datetime.datetime(2022, 3, 1)
end_date = datetime.datetime(2022, 3, 31)

# Generar una lista de fechas dentro del rango
date_list = [ee.Date((start_date + datetime.timedelta(days=i)).strftime('%Y-%m-%d')) for i in range((end_date - start_date).days + 1)]

# Crear una colección de imágenes
collection = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_CO') \
    .select('CO_column_number_density') \
    .filterDate(date_list[0], date_list[-1].advance(1, 'day')) \
    .map(lambda img: img.clip(geometry))  # Recorta las imágenes utilizando la geometría

# Visualización de la imagen
band_viz = {
    'min': 0,
    'max': 0.05,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

# Crear un objeto Mapa interactivo
mapa = geemap.Map()

# Agregar la capa de CO al mapa
mapa.addLayer(collection.mean(), band_viz, 'S5P CO - Marzo 2022')  # agrega la media de la colección de imágenes

# Mostrar el mapa interactivo
mapa


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…